In [1]:
import numpy as np
import requests
import re
import copy
from collections import Counter

# Get Input

In [3]:
sessionId = "os.environ["ADVENT_OF_CODE_SESSION_ID"]"

In [4]:
r = requests.get("https://adventofcode.com/2018/day/7/input", cookies={"session": sessionId})

In [5]:
r.content

b'Step G must be finished before step T can begin.\nStep L must be finished before step V can begin.\nStep D must be finished before step P can begin.\nStep J must be finished before step K can begin.\nStep N must be finished before step B can begin.\nStep K must be finished before step W can begin.\nStep T must be finished before step I can begin.\nStep F must be finished before step E can begin.\nStep P must be finished before step O can begin.\nStep X must be finished before step I can begin.\nStep M must be finished before step S can begin.\nStep Y must be finished before step O can begin.\nStep I must be finished before step Z can begin.\nStep V must be finished before step Z can begin.\nStep Q must be finished before step Z can begin.\nStep H must be finished before step C can begin.\nStep R must be finished before step Z can begin.\nStep U must be finished before step S can begin.\nStep E must be finished before step Z can begin.\nStep O must be finished before step W can begin.

In [9]:
def parse_instructions(line):
    m = re.search('Step ([A-Z]).*step ([A-Z]).*', line.decode('utf-8'))
    return m.group(1), m.group(2)

In [11]:
edges = [parse_instructions(line) for line in r.content.splitlines()]
edges

[('G', 'T'),
 ('L', 'V'),
 ('D', 'P'),
 ('J', 'K'),
 ('N', 'B'),
 ('K', 'W'),
 ('T', 'I'),
 ('F', 'E'),
 ('P', 'O'),
 ('X', 'I'),
 ('M', 'S'),
 ('Y', 'O'),
 ('I', 'Z'),
 ('V', 'Z'),
 ('Q', 'Z'),
 ('H', 'C'),
 ('R', 'Z'),
 ('U', 'S'),
 ('E', 'Z'),
 ('O', 'W'),
 ('Z', 'S'),
 ('S', 'C'),
 ('W', 'B'),
 ('A', 'B'),
 ('C', 'B'),
 ('L', 'P'),
 ('J', 'V'),
 ('E', 'W'),
 ('Z', 'W'),
 ('W', 'C'),
 ('S', 'W'),
 ('Q', 'S'),
 ('O', 'B'),
 ('R', 'W'),
 ('D', 'H'),
 ('E', 'O'),
 ('Y', 'H'),
 ('V', 'O'),
 ('O', 'S'),
 ('X', 'V'),
 ('R', 'E'),
 ('S', 'A'),
 ('K', 'Y'),
 ('V', 'W'),
 ('U', 'W'),
 ('H', 'R'),
 ('P', 'I'),
 ('E', 'C'),
 ('H', 'Z'),
 ('N', 'V'),
 ('N', 'W'),
 ('A', 'C'),
 ('V', 'E'),
 ('N', 'Q'),
 ('Y', 'V'),
 ('R', 'O'),
 ('R', 'C'),
 ('L', 'S'),
 ('V', 'R'),
 ('X', 'R'),
 ('Z', 'A'),
 ('O', 'Z'),
 ('U', 'C'),
 ('X', 'W'),
 ('K', 'O'),
 ('O', 'A'),
 ('K', 'T'),
 ('N', 'O'),
 ('X', 'C'),
 ('Z', 'C'),
 ('N', 'X'),
 ('T', 'A'),
 ('D', 'O'),
 ('M', 'Q'),
 ('D', 'C'),
 ('U', 'E'),
 ('N', 'H'),

# Part 1

In [12]:
import networkx as nx

In [14]:
G = nx.DiGraph()
G.add_edges_from(edges)

In [27]:
def get_source_nodes(G):
    source_nodes = []
    return [n for n, in_deg in G.in_degree() if in_deg == 0]
        

In [28]:
get_source_nodes(G)

['G', 'L', 'J', 'N']

In [32]:
work_order = []
source_nodes = set(get_source_nodes(G))
while source_nodes:
    next_ = sorted(source_nodes)[0]
    work_order.append(next_)
    G.remove_node(next_)
    source_nodes = set(get_source_nodes(G))
    

In [34]:
"".join(work_order)

'GJKLDFNPTMQXIYHUVREOZSAWCB'

# Part 2

In [171]:
short_input = """Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin."""

In [172]:
def parse_instructions_short(line):
    m = re.search('Step ([A-Z]).*step ([A-Z]).*', line)
    return m.group(1), m.group(2)

In [173]:
short_edges = [parse_instructions_short(line) for line in short_input.splitlines()]
short_edges

[('C', 'A'),
 ('C', 'F'),
 ('A', 'B'),
 ('A', 'D'),
 ('B', 'E'),
 ('D', 'E'),
 ('F', 'E')]

In [174]:
edges = [parse_instructions(line) for line in r.content.splitlines()]

In [186]:
def get_available_work(G, current_work = None):
    if current_work is None:
        current_work = []
    source_nodes = set(get_source_nodes(G))
    available_work = [j for j in sorted(source_nodes, reverse = True) if j not in current_work]
    return available_work
    

In [202]:
edges_ = edges
num_workers = 5
offset = 4
# edges_ = short_edges
# num_workers = 2
# offset = 64

In [205]:
G = nx.DiGraph()
G.add_edges_from(edges_)
#print(G)
time = 0
workers = [(None, 0)]*num_workers
current_work = set()
work_order = []

while len(G):
    available_work = get_available_work(G, current_work)
    workers = [(job,max(w-1,0)) for job, w in workers]
    for i, w in enumerate(workers):
        job,work_remaining = w
        if work_remaining == 0 and job is not None:
            G.remove_node(job)
            current_work.remove(job)
            work_order.append(job)
            job = None
            available_work = get_available_work(G, current_work)
            
        if job is None and len(available_work) > 0:
            job = available_work.pop()
            current_work.add(job)
            work_remaining = ord(job)-offset
        
        workers[i] = (job, work_remaining)

    if len(G) == 0:
        break
    else:
        time+=1
time

967

In [198]:
work_order

[]

In [177]:
ord("A")-4

61